In [1]:
import pandas as pd
import os
import gc

# --- Paths for CTU-13 dataset ---
dataset_files = {
    '1': 'scenario_dataset_1/dataset_result.binetflow',
    '2': 'scenario_dataset_2/dataset_result.binetflow',
    '5': 'scenario_dataset_5/dataset_result.binetflow',
    '9': 'scenario_dataset_9/dataset_result.binetflow',
    '13': 'scenario_dataset_13/dataset_result.binetflow'
}
# --- End paths ---

# Tentukan nama folder untuk menyimpan hasil sampel
output_folder = 'sampled_data'

# Buat dictionary kosong untuk menampung semua DataFrame LENGKAP
# Variabel ini akan 'hidup' setelah sel ini dijalankan
dict_of_dataframes = {}

In [2]:
print("Memulai proses: Membaca data & mengambil sampel...")

# 1. Buat folder output jika belum ada
try:
    os.makedirs(output_folder, exist_ok=True)
    print(f"Folder output '{output_folder}' telah disiapkan.")
except Exception as e:
    print(f"Gagal membuat folder '{output_folder}': {e}")


# 2. Loop, baca, simpan ke dict, ambil sampel, dan simpan sampel
for key, filepath in dataset_files.items():
    try:
        print(f"\n--- Memproses Dataset {key} ---")
        print(f"Membaca file: {filepath}...")
        
        # a. Baca file
        df = pd.read_csv(filepath, low_memory=False)
        total_rows = len(df)
        
        # b. Simpan DataFrame LENGKAP ke dictionary
        # Ini memenuhi permintaan Anda untuk mengaksesnya nanti
        dict_of_dataframes[key] = df
        print(f"Dataset {key} ({total_rows} baris) telah dimuat ke memori.")

        # c. Tentukan jumlah sampel
        sample_size = min(100, total_rows)
        if sample_size < 100:
            print(f"Peringatan: File {key} hanya memiliki {total_rows} baris. Mengambil {sample_size} baris.")
        else:
            print(f"Mengambil 100 baris acak dari file {key}...")

        # d. Ambil sampel dari DataFrame yang baru saja dibaca
        sampled_df_botnet_not_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=25, random_state=42)

        sampled_df_botnet_and_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=25, random_state=42)

        sampled_df_no_botnet_no_spam = df[
            ~df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=50, random_state=42)

        # Gabungkan ketiga DataFrame menjadi satu
        sampled_df = pd.concat([sampled_df_botnet_not_spam, sampled_df_botnet_and_spam, sampled_df_no_botnet_no_spam], ignore_index=True)

        # Acak urutan baris
        sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Hapus ketiga variabel yang sudah digabungkan agar tidak memenuhi memori
        del sampled_df_botnet_not_spam
        del sampled_df_botnet_and_spam
        del sampled_df_no_botnet_no_spam

        # Memastikan variabel-variabel sudah dihapus dan memori lebih efisien
        gc.collect()

        # e. Tentukan nama file output dan simpan sampel
        original_folder = os.path.dirname(filepath)  # Dapatkan folder asli dari path file
        folder_name = os.path.basename(original_folder)  # Ambil nama folder seperti '1', '2', dll.
        
        # Buat folder di dalam sampled_data dengan nama folder asal
        folder_output_path = os.path.join(output_folder, folder_name)
        os.makedirs(folder_output_path, exist_ok=True)

        base_filename = os.path.basename(filepath)  # Ambil nama file asli
        output_filepath = os.path.join(folder_output_path, base_filename)  # Gabungkan dengan folder output
        
        sampled_df.to_csv(output_filepath, index=False)
        print(f"Sukses! Sampel disimpan ke '{output_filepath}'")
        
    except FileNotFoundError:
        print(f"PERINGATAN: File tidak ditemukan di '{filepath}'. Dataset {key} akan dilewati.")
    except Exception as e:
        print(f"Gagal memproses file {filepath}: {e}. Dataset {key} akan dilewati.")

print("\n--- Semua proses selesai ---")
print(f"Berhasil memuat {len(dict_of_dataframes)} dataset ke memori.")

Memulai proses: Membaca data & mengambil sampel...
Folder output 'sampled_data' telah disiapkan.

--- Memproses Dataset 1 ---
Membaca file: scenario_dataset_1/dataset_result.binetflow...
Dataset 1 (2112224 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 1...
Sukses! Sampel disimpan ke 'sampled_data\scenario_dataset_1\dataset_result.binetflow'

--- Memproses Dataset 2 ---
Membaca file: scenario_dataset_2/dataset_result.binetflow...
Dataset 2 (1465182 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 2...
Sukses! Sampel disimpan ke 'sampled_data\scenario_dataset_2\dataset_result.binetflow'

--- Memproses Dataset 5 ---
Membaca file: scenario_dataset_5/dataset_result.binetflow...
Dataset 5 (92917 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 5...
Sukses! Sampel disimpan ke 'sampled_data\scenario_dataset_5\dataset_result.binetflow'

--- Memproses Dataset 9 ---
Membaca file: scenario_dataset_9/dataset_result.binetflow...
Dataset 9 (157330

In [3]:
# Cek key (dataset) apa saja yang tersedia di dictionary
print(f"Dataset yang berhasil dimuat ke memori: {list(dict_of_dataframes.keys())}")

# Contoh: Mengakses dan menampilkan 5 baris pertama dari dataset '5' (yang LENGKAP)
try:
    print("\nMenampilkan .head() dari dataset '5' (diambil dari memori):")
    
    # Ini adalah cara Anda mengakses data LENGKAP-nya
    df_5_full = dict_of_dataframes['5']
    
    display(df_5_full.head())
    print(f"Ukuran dataset '5' di memori: {df_5_full.shape}")
    
except KeyError:
    print("Dataset '5' tidak berhasil dimuat (mungkin error atau tidak ditemukan).")

Dataset yang berhasil dimuat ke memori: ['1', '2', '5', '9', '13']

Menampilkan .head() dari dataset '5' (diambil dari memori):


,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2020-01-01 00:00:01,0.000201,udp,147.32.86.168,49808,<->,147.32.80.9,53,CON,0.0,0.0,2,271,75,flow=To-Background-UDP-CVUT-DNS-Server
1,2020-01-01 00:00:02,26.997431,udp,147.32.84.118,1151,->,213.181.217.12,2365,INT,0.0,NaN,2,286,286,flow=Background-UDP-Attempt
2,2020-01-01 00:00:02,0.000186,udp,147.32.85.34,51710,<->,147.32.80.9,53,CON,0.0,0.0,2,323,85,flow=To-Background-UDP-CVUT-DNS-Server
3,2020-01-01 00:00:02,0.000470,udp,99.71.173.50,27406,<->,147.32.84.229,13363,CON,0.0,0.0,2,132,72,flow=Background-UDP-Established
4,2020-01-01 00:00:02,0.000286,udp,147.32.84.164,51680,<->,147.32.80.9,53,CON,0.0,0.0,2,201,72,flow=From-Normal-V46-Grill


Ukuran dataset '5' di memori: (92917, 15)
